In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
try:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
    from tensorflow.keras.utils import to_categorical
except ModuleNotFoundError as e:
    raise ImportError("TensorFlow is not installed in your environment. Please install TensorFlow using 'pip install tensorflow' and try again.") from e

# Define paths
data_path = "C:\\Users\\Admin\\Downloads\\ChestXRay2017\\chest_xray\\train"
classes = ['NORMAL', 'PNEUMONIA']

# Function to preprocess images
def preprocess_image(img):
    img_resized = cv2.resize(img, (128, 128))  # Resize to 128x128
    negative_img = 255 - img_resized            # Convert to negative image

    # Convert to heatmap
    heatmap_img = cv2.applyColorMap(cv2.normalize(img_resized, None, 0, 255, cv2.NORM_MINMAX), cv2.COLORMAP_JET)
    heatmap_img = cv2.cvtColor(heatmap_img, cv2.COLOR_BGR2GRAY)  # Convert heatmap to grayscale

    return img_resized, negative_img, heatmap_img

# Load and preprocess data
data = []
labels = []
for label in classes:
    class_path = os.path.join(data_path, label)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img_resized, negative_img, heatmap_img = preprocess_image(img)
            data.append(np.expand_dims(img_resized, axis=-1))  # Add channel dimension
            data.append(np.expand_dims(negative_img, axis=-1))
            data.append(np.expand_dims(heatmap_img, axis=-1))
            labels.extend([classes.index(label)] * 3)

# Convert data and labels to numpy arrays
data = np.array(data, dtype=np.float32) / 255.0  # Ensure consistent dtype
labels = to_categorical(np.array(labels), num_classes=len(classes))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    epochs=20)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Generate classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=classes))

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()

# Save the trained model
model.save('pneumonia_model.h5')
print("Model saved as pneumonia_model.h5")

